 <h1 align="center" style="font-size:200%; color:blue">External data sources</h1><br>

As the initial data given were of poor quality alternative sources had to be searched. Following 3 sources were the most readily available and of high quality
* IMBD free API
* IMDBpro data search with paid subscription along with web scraping
* TMDB free API

In [1]:
import pandas as pd
import numpy as np
import os
import time
from csv import writer
from bs4 import BeautifulSoup
from imdb import IMDb

# IMDB API 
direct from IMDB and registering is not required.  
This was working well on testing but due to forgetting to put a delay in the loop I was blocked out of IMDB

In [2]:
#movies=pd.read_csv('revenue.csv')
#movies['rank'] = movies['rank'].str.replace(',', '')
#movies['rank'] = movies['rank'].astype('int64')
###print(movies.head(2))
###print(movies.info())

In [3]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem) 

In [4]:
def getIMDB_info(tconst):
    # create an instance of the IMDb class
    ia = IMDb()
    
    # get a movie
    movie = ia.get_movie(tconst.replace("tt",""))
    # Wait for 5 seconds
    time.sleep(4)

    # return language of the movie
    movie_lang=''
    for lang in movie['language']:
        movie_lang=movie_lang+","+lang

    # return country of the movie
    movie_country=movie['country']
    
    # print the genres of the movie
    movie_genres = ''
    for genre in movie['genres']:
        movie_genres=movie_genres+","+genre

    # print the runtime of the movie
    movie_mins = str(movie['runtime'][0])

    # print the averageRating of the movie
    movie_rate=str(movie['rating'])

    return movie_lang, movie_country, movie_genres, movie_mins, movie_rate

In [5]:
#movies2 = movies.loc[0:10]

#movies2['language'] = movies2['imdb_id'].map(lambda x: getIMDB_info(x))
#print(movies2)

#for index, rows in movies2.iterrows():
#    lang, country, genres, mins, rate  = getIMDB_info(rows.imdb_id)
#    ### str(rows.rank) # there is a serious problem with this
#    mylist=[rows.imdb_id, rows.title, rows.year, rows.director, rows.production_co, rows.budget_usd, rows.us_gross, lang, country, genres, mins, rate]
#    
#    print(mylist)
#    append_list_as_row('IMDB_download.csv', mylist)

# IMDBpro
Credits: Jesse Neuman developed webscraping code
An API was not offered. This web-scraping is neededlogin is required. Normal scraping is not easy. 

1. search result loaded
       search results are lazy loaded and scrolling down manually is quite time consuming. This Jesse developed the following  java script on console to automate the scrolling

var lastScrollHeight = 0;
function autoScroll() {
  var sh = document.documentElement.scrollHeight;
  if (sh != lastScrollHeight) {
    lastScrollHeight = sh;
    document.documentElement.scrollTop = sh;
  }
}
var id = window.setInterval(autoScroll, 100);

once it's scrolled to the bottom the page is saved locally so that scraping can be done locally avoiding the login issue. 

## parse the saved html file fom BeautifulSoup's HTML parser

In [6]:
def read_html(file_path):
    """Takes a saved html page and returns the results container that holds all the targeted info

    Arguments:
        file_path (string): path to find the saved html page

    Returns:
        result (BeautifulSoup object): or None if result wasn't found
    """
    # strip all the actor elements from the first html
    with open(file_path, 'r', encoding='utf8') as f_html:
        html = BeautifulSoup(f_html, "html.parser")
        # pull out only the elements we want
        result = html.find('div', id='results').ul.find_all(
            'li', recursive=False)
        return result if result else None

In [7]:
# setup paths for importing raw html
#movie_html_a_path = os.path.join('Data', 'IMDbPro_search.html')
#print(movie_html_a_path)
#movie_html_b_path = os.path.join(
#    os.pardir, 'data', 'external', 'movie_money_1m_minus.htm')

In [8]:
money_a = read_html('../SavedHTML/IMDbPro_search.html') # this DB is too big for GitHub. Had to take out of repo

In [9]:
# sanity check the results
print('type of container 1: {}'.format(
    type(money_a)))
print('size of container 1: {}'.format(
    len(money_a)))

type of container 1: <class 'bs4.element.ResultSet'>
size of container 1: 10001


In [10]:
# container parser
def money_parser(soup):
    """Parses through an array of soup objects and takes out the relevant info

    Arguments:
        soup (bs4.element.ResultSet): Chunk to search through

    Returns:
        results (List): A list of dictionaries 
    """
    results = []
    i = 0
    for title in soup:
        result = {
            'imdb_id': title.find('span', class_='display-title').a.get('href')[27:36] if title.find('span', class_='display-title') else None,
            'title': title.find('span', class_='display-title').a.get_text() if title.find('span', class_='display-title') else None,
            'year': title.find('span', class_='year').get_text()[1:-1] if title.find('span', class_='year') else None,
            'director': title.find('span', class_='display-name').a.get_text() if title.find('span', class_='display-name') else None,
            'production_co': title.find('span', class_='display-company').a.get_text() if title.find('span', class_='display-company') else None,
            'region_code': title.find('span', class_='region_code').get_text().strip() if title.find('span', class_='region_code') else None,
            'rank': title.find('span', class_='ranking').get_text().strip() if title.find('span', class_='ranking') else None,
            'budget_usd': title.find('span', class_='budget_usd').get_text().strip() if title.find('span', class_='budget_usd') else None,
            'us_gross': title.find('span', class_='us_gross').get_text().strip() if title.find('span', class_='us_gross') else None
        }

        for li in title.find_all("li"):
            li_txt = li.get_text().strip()
            # 'Country' & 'Countries' difficult to get!!!
            #if 'Country' in li_txt:
            #    c=li_txt.replace('Country:', '').strip()
            #    #print(c)
            #    result['Country'] = c
            if 'Genre' in li_txt:
                g=li_txt.replace('Genre:', '').strip()
                g=g.replace(' |', ',')
                #print(g)
                result['genres'] = g
            else:
                result['genres'] = None
        #if i >3  :  # unit test 
        #    break
        #i += 1
        results.append(result)
    return results

#money_parser(money_a)

In [11]:
parsed_money = money_parser(money_a)

In [12]:
parsed_money[8800]

{'imdb_id': 'tt0216651',
 'title': 'Vampire Hunter D: Bloodlust',
 'year': '2000',
 'director': 'Yoshiaki Kawajiri',
 'production_co': 'BMG Funhouse',
 'region_code': '[JP]',
 'rank': '12,782',
 'budget_usd': '',
 'us_gross': '$151K',
 'genres': 'Animation, Action, Fantasy'}

## inspect parsed data to confirm successful extraction 

In [13]:
# load up new dataset with the parsed info
money_test_df = pd.DataFrame.from_dict(parsed_money)
# better to clean a little here before saving .csv
money_test_df['year'].fillna(value=-1.0,inplace=True)
money_test_df['year'] = money_test_df['year'].astype(int)
money_test_df['rank'] = money_test_df['rank'].str.replace(',', '')
money_test_df['rank'].fillna(value=-1.0,inplace=True)
money_test_df['rank'] = money_test_df['rank'].astype(int)

In [14]:
money_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 10 columns):
imdb_id          10000 non-null object
title            10000 non-null object
year             10001 non-null int32
director         9999 non-null object
production_co    9926 non-null object
region_code      9551 non-null object
rank             10001 non-null int32
budget_usd       10000 non-null object
us_gross         10000 non-null object
genres           9999 non-null object
dtypes: int32(2), object(8)
memory usage: 703.3+ KB


In [15]:
# look at the description
money_test_df.describe()

,year,rank
count,10001.000000,10001.000000
mean,2001.047995,7237.469953
std,27.299918,4138.127952
min,-1.000000,-1.000000
25%,1992.000000,3640.000000
50%,2007.000000,7140.000000
75%,2016.000000,10776.000000
max,2020.000000,14622.000000


In [16]:
# look at some samples
money_test_df.sample(3)

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross,genres
686,tt0151804,Office Space,1999,Mike Judge,Twentieth Century Fox,[US],1178,$10MM,$11MM,Comedy
7422,tt7533846,Time of happiness,2017,Senol Sönmez,Nulook Production,[TR],10666,,,"Comedy, Romance"
8796,tt7213936,What Will People Say,2017,Iram Haq,Mer Film,[NO],12777,,$44K,Drama


In [17]:
# test for na values
display(money_test_df['us_gross'].isna().sum())
display(money_test_df['budget_usd'].isna().sum())
display(money_test_df['genres'].isna().sum())

1

1

2

Extracted data looks okay. However, unlike Jesse I got 10,000 records. I will however use this data file and repeate the analysis

In [18]:
# safe as an interim file. After scrubbing it will be saved in final form.
out_path = os.path.join('Data', 'interim_IMDBpro.csv')
money_test_df.to_csv(out_path, index=False)

## initial data scrubbing 

In [19]:
revenue_df = pd.read_csv(out_path)
revenue_df['imdb_id'].sample(3)

7109    tt8652584
7098    tt0104549
9942    tt0094793
Name: imdb_id, dtype: object

In [20]:
revenue_df.head()

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross,genres
0,tt8228288,The Platform,2019,Galder Gaztelu-Urrutia,Basque Films,[ES],1,NaN,NaN,"Horror, Sci-Fi, Thriller"
1,tt1598778,Contagion,2011,Steven Soderbergh,Warner Bros.,[US],2,$60MM,$76MM,"Action, Drama, Thriller"
2,tt1051906,The Invisible Man,2020,Leigh Whannell,Universal Pictures,[US],4,$7MM,$65MM,"Horror, Mystery, Sci-Fi"
3,tt8244784,The Hunt,2020,Craig Zobel,Blumhouse Productions,[US],7,$14MM,$5.8MM,"Action, Horror, Thriller"
4,tt1634106,Bloodshot,2020,Dave Wilson,Sony Pictures Entertainment (SPE),[US],9,$45MM,$10MM,"Action, Drama, Sci-Fi"


In [21]:
revenue_df.loc[revenue_df['imdb_id'] == 'tt0091605']

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross,genres
2618,tt0091605,The Name of the Rose,1986,Jean-Jacques Annaud,Constantin Film,[DE],3817,ITL 30B,$7.2MM,"Crime, Drama, Mystery"


## what processing is needed for scrapped dataset ? :

1. Convert 'us_gross', and 'budget_usd' values into floats. That means stripping the non-number characters out as well as changing 'MM' to ',000,000'.
2. Convert year column to int, so the years don't have the trailing .0.

In [22]:
# Created 3/22/2020 with current exhange values. Values not adjusted for the date the movie was created.
def get_conversion_rate(value):
    """Get exchange rate for given currency code
    
    Arguments:
        value (string): String with currency code or symbol in it

    Returns:
        rate (float): Conversion rate to usd
    """
    if '£' in value:
        return 0.854
    elif '€' in value:
        return 0.9334
    elif 'AUD' in value:
        return 1.7229
    elif 'CAD' in value:
        return 1.435
    elif 'FRF' in value:
        return 6.55957 * 0.9334
    elif 'INR' in value:
        return 75.394
    elif 'THB' in value:
        return 32.68
    elif 'EM' in value:
        return 1 # cant find info on EM
    elif 'JPY' in value:
        return 110.75
    elif 'SKW' in value:
        return 1254.45
    elif 'HUF' in value:
        return 327.94
    elif 'NGN' in value:
        return 364
    elif 'CNY' in value:
        return 7.0950
    elif 'ESP' in value:
        return 155.42826
    elif 'RUR' in value:
        return 79.87
    elif 'HKD' in value:
        return 7.7570
    elif 'ISK' in value:
        return 140.490
    elif 'PHP' in value:
        return 51.19
    elif 'DKK' in value:
        return 6.9716
    elif 'CZK' in value:
        return 25.5620
    elif 'SKK' in value:
        return 10.3753
    elif 'NOK' in value:
        return 11.7890
    elif 'MXN' in value:
        return 24.4215
    elif 'JMD' in value:
        return 135.07
    elif 'PLN' in value:
        return 4.23
    elif 'KRW' in value:
        return 1228.97
    elif 'ITL' in value:
        return 1804.64
    else:
        return 1

In [23]:
def strip_currency_code(value):
    """Strips currency code from front of currency string

    Arguments: 
        value (string): currency amount prefaced with currency code
    
    Returns:
        value (string): value without the currency code
    """
    if value[:1] in '$£€':
        return value[1:]
    else:
        return value[3:]

In [24]:
def convert_money(value):
    """Takes currency string and parses it into correct amount in USD
    
    Arguments:
        value (string): currency in form: CAD 345.3B 

    Returns:
        value (int): currency converted to USD and in standard numeric form
    """
    # type check:
    if type(value) != str:
        return
    
    # check currency sign and get coefficient
    coef = get_conversion_rate(value)
    value = strip_currency_code(value)
    if 'K' in value:
        value = (float(value.strip('K')) * 1000) / coef
    elif 'MM' in value:
        value = (float(value.strip('MM')) * 1000000) / coef
    elif 'B' in value:
        value = (float(value.strip('B')) * 1000000000) / coef
    else:
        value = float(value.strip()) / coef
    return value

In [25]:
revenue_df['us_gross'] = revenue_df['us_gross'].apply(convert_money)

In [26]:
revenue_df['us_gross']

0               NaN
1        76000000.0
2        65000000.0
3         5800000.0
4        10000000.0
            ...    
9996            NaN
9997            NaN
9998      1000000.0
9999      4500000.0
10000           NaN
Name: us_gross, Length: 10001, dtype: float64

In [27]:
revenue_df['budget_usd'] = revenue_df['budget_usd'].apply(convert_money)

In [28]:
revenue_df['budget_usd'].isna().sum()

3493

In [29]:
# first, change the missing values from budget to -1, so we dont drop 1910 rows.
revenue_df['budget_usd'].fillna(value=-1.0,inplace=True)
revenue_df['us_gross'].fillna(value=-1.0,inplace=True)

# also, fill in the production_co missing values with an 'Unknown'
##revenue_df['production_co'] = revenue_df['production_co'].fillna(value='Unknown', inplace=True)
revenue_df.drop(['production_co'], axis = 1, inplace=True) # not using and many null. So dropping 

display(revenue_df.sample(3))
display(revenue_df.info())
display(revenue_df['budget_usd'].isna().sum())

,imdb_id,title,year,director,region_code,rank,budget_usd,us_gross,genres
1019,tt6292852,I Am Mother,2019,Grant Sputore,[AU],1661,-1.0,-1.0,"Drama, Mystery, Sci-Fi"
5371,tt2404181,Belle,2013,Amma Asante,[GB],7658,11000000.0,11000000.0,"Biography, Drama, Romance"
2606,tt8406738,Advantages of Travelling by Train,2019,Aritz Moreno,[ES],3794,-1.0,-1.0,Thriller


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 9 columns):
imdb_id        10000 non-null object
title          10000 non-null object
year           10001 non-null int64
director       9999 non-null object
region_code    9551 non-null object
rank           10001 non-null int64
budget_usd     10001 non-null float64
us_gross       10001 non-null float64
genres         9999 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 703.3+ KB


None

0

In [30]:
revenue_save_path = os.path.join( 'Data', 'revenue.csv')
revenue_df.to_csv(revenue_save_path, index=False)

# The Movie DataBase (TMDB) API

TMBD is another web API that maybe useful in finding the movie information. 

## 3rd party python framework "tmdbsimple"
https://github.com/celiao/tmdbsimple.git
This framework is most intriguing. However, it cannot retrieve movie information via IMDB_id, which is most unique and widely used ID to designate movie information. Following are examples how movie information can be retrieve for this method.  

In [41]:
import tmdbsimple as tmdb
key=str(open('../TMDB_key.txt', 'r').readline()).strip()
tmdb.API_KEY = key

In [45]:
# example of how to retrieve movie information if TMDB ID is provided 
movie = tmdb.Movies(603)
response = movie.info()
print(movie.title)
print(movie.budget)

The Matrix
63000000


In [48]:
search = tmdb.Search()
response = search.movie(query='The Bourne')
for s in search.results:
    print(s['title'], s['id'], s['popularity'])

The Bourne Ultimatum 2503 45.018
The Bourne Identity 2501 18.371
The Bourne Supremacy 2502 17.437
The Bourne Legacy 49040 14.853
Jason Bourne 324668 17.403
Untitled Jeremy Renner/Bourne Sequel 393640 1.96


## Direct HTTP request from TMDB. 
This method works well


In [62]:
import http.client
import json
#conn = http.client.HTTPSConnection("www.python.org")
#conn.request("GET", "/")
#r1 = conn.getresponse()
#print(r1.status, r1.reason)
#data1 = r1.read()
#print(data1)

In [68]:
# TMDB web API queries
conn = http.client.HTTPSConnection("api.themoviedb.org")
imdb_id='tt1043726'
query='/3/find/'+imdb_id+'?api_key='+key+'&language=en-US&external_source=imdb_id'
conn.request("GET", query)
r1 = conn.getresponse()
print(r1.status, r1.reason)
data1 = r1.read()
x=json.loads(data1)
print(x['movie_results'][0]['title'])
print(x['movie_results'][0]['original_language'])
print(x['movie_results'][0]['release_date'])
print(x['movie_results'][0]['vote_average'])

200 OK
The Legend of Hercules
en
2014-01-10
4.6
